In [1]:
%run _BaseInfo.ipynb
import time

from openpyxl import load_workbook
def save_excel(file_name,df):
    sheet_name='Sheet1'
    try:
        # 嘗試加載現有的 Excel 文件
        book = load_workbook(file_name)
        with pd.ExcelWriter(file_name, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            # 加載現有數據
            writer.book = book
            if sheet_name in book.sheetnames:
                existing_df = pd.read_excel(file_name, sheet_name=sheet_name)
                #df = pd.concat([existing_df, df], ignore_index=True)
            # 將 DataFrame 寫入指定的 Sheet
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    except FileNotFoundError:
        # 如果文件不存在，創建新文件並寫入 DataFrame
        with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name=sheet_name, index=False)

In [6]:
df_data_all= pd.DataFrame()
stock_list = _baseInfo.dropna()['公司代號'].iloc[::-1]
for code in stock_list:
    print(code)
    try:
        time.sleep(1)
        url='https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a='+code
        print(url)
        response = requests.get(url,verify = False)
        response.encoding = 'big5'
        df = pd.DataFrame()
        df = pd.concat([df, pd.DataFrame([[str(code)]], columns=['stock_number'])], ignore_index=True)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            table=soup.find('table').find('table').find('table')

            for x in range(len(table.find_all('tr'))):
                #if x>=1 and x<5:
                if x!=0  and (x<12 or x==20) :
                    detail=[th.text.replace('', '') for th in table.find_all('tr')[x].find_all('td')]
                    for y in range(len(detail)-1):
                        if (y % 2) == 0:
                            df[detail[y]]=str(detail[y+1])

            save_df=df#[['stock_number', '收盤價' ,'每股淨值(元)','股價淨值比','本益比', '一年內最高價', '一年內最低價', '一年內最大量', '一年內最低量','營收比重']]
            df_data_all=pd.concat([ df_data_all,save_df], axis=0)

    except:
            print('error'+code)

9962
https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a=9962


In [7]:
df_data_all

,stock_number,開盤價,最高價,最低價,收盤價,漲跌,一年內最高價,一年內最低價,本益比,一年內最大量,一年內最低量,成交量,同業平均本益比,一年來最高本益比,一年來最低本益比,盤後量,殖利率,總市值,85年來最高總市值,85年來最低總市值,投資報酬率(12/06),獲利能力(113.3Q),今年以來,每股淨值(元),營業毛利率,貝他值,最近一週,每人營收(仟元),營業利益率,標準差,最近一個月,每股營收(元),稅前淨利率,最近二個月,負債比例,資產報酬率,最近三個月,股價淨值比,股東權益報酬率,營收比重
0,9962,15.6,15.7,15.6,15.6,-0.05,19.55,14,22.29,"4,148",13,53,23.28,22.36,10.12,0,7.05%,"1,407","3,457",363,財務比例(113.3Q),投資風險(12/06),-9.13%,12.18,8.06%,0.43,0.97%,N/A,4.14%,1.17%,0.65%,6.42,3.68%,-5.74%,27.81%,1.13%,2.63%,1.28,1.50%,不�袗�板78.84%、不�袗�料20.94%、客戶委託代工(OEM)0.22% (2023年)


In [5]:
df_data_all
file_name='Result/StockInfo.xlsx'
#save_excel(file_name,df_data_all)
df_data_all.to_excel(file_name, index=False)